In [110]:
from keras.datasets import cifar10
from keras import layers,models
from keras.layers import Conv2D,Dense,MaxPooling2D,Flatten
import keras
import numpy as np

In [4]:
import keras
from keras.datasets import cifar10
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from keras import models
import numpy as np

In [5]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [6]:
(x_train_row,y_train_row),(x_test_row,y_test_row) = cifar10.load_data()
print(x_train_row.shape)
print(y_train_row.shape)


(50000, 32, 32, 3)
(50000, 1)


In [18]:
def x_process(x,flatten = True):
    x = x / 255
    if flatten:
        x = x.reshape(x.shape[0],-1)
    print(x.shape)
    return x

def y_process(y):
    y =keras.utils.to_categorical(y)
    print(y.shape)
    return y

In [21]:
x_train = x_process(x_train_row,False)
x_test = x_process(y_train_row,False)
y_train = y_process(x_test_row)
y_test = y_process(y_test_row)
print(y_test[0])

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [9]:
def build_Conv2D(filters = [64,64,64]):
    model = keras.models.Sequential()
    for filter_num in filters:
        model.add(Conv2D(filter_num,(3,3),activation = 'relu',padding = 'same',input_shape = x_train.shape[1:]))
        model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(units = 512,activation = 'relu'))
    model.add(Dense(units = 10,activation = 'softmax'))
    return model
    
'''
    input_layer = keras.layers.Input([x_train.shape[-1]])
    for index,filter_num in enumerate(filters[1:]):
        if(index == 0):
            x = layers.Conv2D(filter_num,(3,3),activation = 'relu',padding = 'same')(input_layer)
        else:
            x = layers.Conv2D(filter_num,(3,3),activation = 'relu',padding = 'same')(x)
        x = layers.Maxpooling(2,2)(x)
        
    x = layers.Flatten()(x)
    x = layers.Dense(units = 512,activation = 'relu')(x)
    output_layer = layers.Dense(units = 10,activation = 'softmax')
    model = models.Model(inputs = input_layer,outputs = output_layer)
'''

"\n    input_layer = keras.layers.Input([x_train.shape[-1]])\n    for index,filter_num in enumerate(filters[1:]):\n        if(index == 0):\n            x = layers.Conv2D(filter_num,(3,3),activation = 'relu',padding = 'same')(input_layer)\n        else:\n            x = layers.Conv2D(filter_num,(3,3),activation = 'relu',padding = 'same')(x)\n        x = layers.Maxpooling(2,2)(x)\n        \n    x = layers.Flatten()(x)\n    x = layers.Dense(units = 512,activation = 'relu')(x)\n    output_layer = layers.Dense(units = 10,activation = 'softmax')\n    model = models.Model(inputs = input_layer,outputs = output_layer)\n"

In [10]:
def data_generator(train,test,batch_size = 32):
    for index in range(0,len(train),batch_size):
        image = train[index:index+batch_size]
        label = test[index:index+batch_size]
        yield image,label

In [24]:
from keras.preprocessing.image import ImageDataGenerator
Datagen = ImageDataGenerator()
train_flow = Datagen.flow(x_train,y_train)
test_flow = Datagen.flow(x_test,y_test)

In [25]:
model = build_Conv2D()
optimizer = keras.optimizers.SGD(lr = 1e-3,momentum = .95,nesterov = True)
model.compile(loss = 'categorical_crossentropy',metrics = ['accuracy'],optimizer = optimizer)
steps = x_train.shape[0] / 32
model.fit_generator(train_flow,
                    steps_per_epoch = steps,
                    epochs = 2,
                   validation_data = test_flow,
                   validation_steps = steps)

Epoch 1/2
1563/1562 [==============================] - 35s 22ms/step - loss: 1.8147 - accuracy: 0.3399 - val_loss: 1.3730 - val_accuracy: 0.4412
Epoch 2/2
1563/1562 [==============================] - 37s 23ms/step - loss: 1.3825 - accuracy: 0.5051 - val_loss: 1.1258 - val_accuracy: 0.5545
